# Simple Agent with Microsoft Agent Framework

This notebook demonstrates a simple agent using the Microsoft Agent Framework with Microsoft Foundry (V2 SDK) and Neo4j.

The code:

- Creates an agent using `AzureAIClient` connected to Microsoft Foundry
- Connects to your Neo4j database
- Defines a `get_graph_schema` tool
- Streams the agent's response

Review and run the code and answer the following questions: 

1. What is the agent's function? 
2. What can it do?
3. How could you extend it?

***

Load the environment variables and import the required Python modules.

In [ ]:
import sys
sys.path.insert(0, '../new-workshops/solutions')

from neo4j_graphrag.schema import get_schema

from agent_framework.azure import AzureAIClient
from azure.identity.aio import AzureCliCredential

from config import get_neo4j_driver, get_agent_config

Get the agent configuration and connect to your Neo4j `graph` database.

In [ ]:
# Get configuration from environment
config = get_agent_config()

# Connect to Neo4j using context manager
driver = get_neo4j_driver().__enter__()

Create the `tools` your agent will use.

In the Microsoft Agent Framework, tools are defined as simple Python functions with docstrings. The agent will use the function name and docstring to determine when to execute the tool.

In [ ]:
# Define the schema tool
def get_graph_schema() -> str:
    """Get the schema of the graph database including node labels, relationships, and properties."""
    return get_schema(driver)

# Define a list of tools for the agent
tools = [get_graph_schema]

> The agent will use the tool's name (`get_graph_schema`) and docstring (`Get the schema of the graph database...`) to determine whether it should execute the tool to resolve a user's query.

***

Create the agent using the Microsoft Agent Framework with Microsoft Foundry.

The `AzureAIClient` connects to your Microsoft Foundry project and creates a persistent agent.

In [ ]:
# Create credential and client
credential = AzureCliCredential()

client = AzureAIClient(
    project_endpoint=config.project_endpoint,
    model_deployment_name=config.model_name,
    async_credential=credential,
)

Create a query for the agent, create the agent with tools, and stream the messages.

When you run the agent, you will see:

1. The messages between the user and the agent
2. The context of the database schema (retrieved by the tool)
3. The agent's final response

In [ ]:
# Run the agent
query = "Summarise the schema of the graph database."

async def run_agent():
    async with client.create_agent(
        name="workshop-schema-agent",
        instructions="You are a helpful assistant that can answer questions about a graph database schema.",
        tools=tools,
    ) as agent:
        print(f"User: {query}\n")
        print("Assistant: ", end="", flush=True)
        
        async for update in agent.run_stream(query):
            if update.text:
                print(update.text, end="", flush=True)
        
        print("\n")

await run_agent()

Experiment with the agent, modify the `query` to ask different questions, for example:

* `"How are Products related to other entities?"`
* `"What questions can I answer using this graph database?"`
* `"How does the graph model relate financial documents to risk factors?"`

***

[View the complete code](../new-workshops/solutions/03_01_simple_agent.py)

[Move on to the Vector + Graph Agent Notebook](03_02_vector_graph_agent.ipynb)

In [ ]:
# Try a different query
query = "What questions can I answer using this graph database?"

async def run_experiment():
    # Create a fresh client for this experiment
    credential = AzureCliCredential()
    client = AzureAIClient(
        project_endpoint=config.project_endpoint,
        model_deployment_name=config.model_name,
        async_credential=credential,
    )
    
    async with client.create_agent(
        name="workshop-schema-agent",
        instructions="You are a helpful assistant that can answer questions about a graph database schema.",
        tools=tools,
    ) as agent:
        print(f"User: {query}\n")
        print("Assistant: ", end="", flush=True)
        
        async for update in agent.run_stream(query):
            if update.text:
                print(update.text, end="", flush=True)
        
        print("\n")
    
    await credential.close()

await run_experiment()

In [ ]:
# Cleanup
driver.close()
await credential.close()